In [1]:
import configparser
import praw
from confluent_kafka import Producer
import socket
import json

In [2]:
# Read passwords and secrets from config file
config_parser = configparser.ConfigParser()
config_parser.read("src/configuration/config.cfg")

['src/configuration/config.cfg']

In [3]:
# Set parameters
reddit_client_id = config_parser["praw"]["client_id"]
reddit_client_secret = config_parser["praw"]["client_secret"]
reddit_password = config_parser["praw"]["password"]
reddit_username = config_parser["praw"]["username"]
reddit_agent = config_parser["praw"]["user_agent"] + reddit_username
sub_reddit = config_parser["praw"]["subreddit"]

In [4]:
# Establish connection to reddit
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    password=reddit_password,
    user_agent=reddit_agent,
    username=reddit_username,
)

In [5]:
conf = {'bootstrap.servers': "broker:29092",
        'client.id': socket.gethostname()}

producer = Producer(conf)

In [ ]:
# Connect to the desired subreddit and start the comments producer
subreddit = reddit.subreddit(sub_reddit)

for comment in subreddit.stream.comments():
    str_comment = str(vars(comment))
    json_comment = json.dumps(str_comment)
    data = json.loads(json_comment)
    producer.produce("comments", key=comment.id, value=data)